In [3]:
import jax.numpy as jnp
import numpyro

/home/dulunche/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
x = [1,2,3]
z = [2,3,4]

In [37]:
x = jnp.array(x)
z = jnp.array(z)

In [38]:
x = x.reshape(x.shape[0],1)
z = z.reshape(z.shape[0],1)

In [39]:
n_x, m = x.shape
n_z, m_z = z.shape

In [40]:
print(n_x, n_z, m, m_z)

3 3 1 1


In [41]:
assert m == m_z

In [64]:
delta = jnp.zeros((n_x, n_z))

In [65]:
delta

DeviceArray([[0., 0., 0.],
             [0., 0., 0.],
             [0., 0., 0.]], dtype=float32)

In [66]:
for d in jnp.arange(m):
    print(d)
    x_d = x[:,d]
    z_d = z[:,d]
    print(x_d, z_d)
    print(x_d[:,jnp.newaxis])
    delta += (x_d[:,jnp.newaxis] - z_d)**2
    print(delta)
jnp.sqrt(delta)

0
[1 2 3] [2 3 4]
[[1]
 [2]
 [3]]
[[1. 4. 9.]
 [0. 1. 4.]
 [1. 0. 1.]]


DeviceArray([[1., 2., 3.],
             [0., 1., 2.],
             [1., 0., 1.]], dtype=float32)

In [45]:
def dist_euclid(x, z):
    x = jnp.array(x) 
    z = jnp.array(z)
    if len(x.shape)==1:
        x = x.reshape(x.shape[0], 1)
    if len(z.shape)==1:
        z = x.reshape(x.shape[0], 1)
    n_x, m = x.shape
    n_z, m_z = z.shape
    assert m == m_z
    delta = jnp.zeros((n_x,n_z))
    for d in jnp.arange(m):
        x_d = x[:,d]
        z_d = z[:,d]
        delta += (x_d[:,jnp.newaxis] - z_d)**2
    return jnp.sqrt(delta)

In [67]:
dist_euclid([1,2,3], [2,3,4])

DeviceArray([[0., 1., 2.],
             [1., 0., 1.],
             [2., 1., 0.]], dtype=float32)

In [46]:
def exp_sq_kernel(x, z, var, length, noise=0, jitter=1.0e-6):
    dist = dist_euclid(x, z)
    deltaXsq = jnp.power(dist/ length, 2.0)
    k = var * jnp.exp(-0.5 * deltaXsq)
    k += (noise + jitter) * jnp.eye(x.shape[0])
    return k